# Text Mining and ML example Example

We have a dataframe with tweets from both members of the Republican as well as the Democrat Party. We would like to analyse which party members cite more clikcbait articles. This could give an indication about the type of articles they or their voters like to consume. For this we first need to label the articles headers as clikbait or non clickbait using Logistic regression which we train on a dataset with article headers where we aleady know if they are clickbiat or not. Then we can compare the clikcbait artice use between democrats and republicans taking into account their total amount of tweets and look at potential factors why one party uses more clickbait articles than the other.



In [ ]:
#First I import all the necessary packages. In this case
# pandas to work with a dataframe
#pickle since we are dealing with a .p file and we need this package to upload such a file
#train_test_split to split our data into training and test data
#CountVectorizer to vectorize our text and to tokenize it which is necessary before performing logistic regression 
#from sklearn.linear_model import LogisticRegression to train our model to be able to receive a function that predicts the label (non-clickbait, clickbait) of our test set
#chi2_contingency to form a contigency table for a later chi square test


In [ ]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

!pip install scipy
from scipy.stats import chi2_contingency 



In [ ]:
#I ,then ,upload the framing. p file using pickle and define this as my dataframe. Here, I need to specify the path of the file as it is not my working directory.

df = pd.read_pickle(r"C:\Users\Luitwin\Desktop\Master Dat Science\Data Mining\week 2/framing.p")

In [ ]:
#For a first overvierw I print the dataframe.

df

,tweet_id,date,user,party,state,chamber,tweet,news_mention,url_reference,netloc,title,description,label
0,1325914751495499776,2020-11-09 21:34:45,SenShelby,R,Alabama,Senator,ICYMI – @BusinessInsider declared #Huntsville ...,businessinsider,https://www.businessinsider.com/personal-finan...,www.businessinsider.com,The 10 best US cities to move to if you want t...,The best US cities to move to if you want to s...,
1,1294021087118987264,2020-08-13 21:20:43,SenShelby,R,Alabama,Senator,Great news! Today @mazda_toyota announced an a...,,https://pressroom.toyota.com/mazda-and-toyota-...,pressroom.toyota.com,Mazda and Toyota Further Commitment to U.S. Ma...,"HUNTSVILLE, Ala., (Aug. 13, 2020) – Today, Maz...",
2,1323340848130609156,2020-11-02 19:06:59,DougJones,D,Alabama,Senator,He’s already quitting on the folks of Alabama ...,,https://apnews.com/article/c73f0dfe8008ebaf85e...,apnews.com,"Tuberville, Jones fight for Senate seat in Ala...","GARDENDALE, Ala. (AP) — U.S. Sen. Doug Jones, ...",
3,1323004075831709698,2020-11-01 20:48:46,DougJones,D,Alabama,Senator,I know you guys are getting bombarded with fun...,,https://secure.actblue.com/donate/djfs-close?r...,secure.actblue.com,I just gave!,Join us! Contribute today.,negiotated
4,1322567531320717314,2020-10-31 15:54:06,DougJones,D,Alabama,Senator,"Well looky here folks, his own players don’t t...",,https://slate.com/culture/2020/10/tommy-tuberv...,slate.com,What Tommy Tuberville’s Former Auburn Players ...,"""All I could think is, why?""",
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30803,1305999044196675584,2020-09-15 22:36:51,Liz_Cheney,R,Wyoming At-Large,U.S. Representative,EXCLUSIVE: Liz Cheney Asks The DOJ To Investig...,,https://dailycaller.com/2020/09/15/liz-cheney-...,dailycaller.com,EXCLUSIVE: Liz Cheney Asks The DOJ To Investig...,Rep. Cheney asked the DOJ to probe whether Rus...,
30804,1301257687049687041,2020-09-02 20:36:23,Liz_Cheney,R,Wyoming At-Large,U.S. Representative,Hairpocrisy!\n#PelosiBlowout \n\nNancy Pelosi ...,,https://www.washingtontimes.com/news/2020/sep/...,www.washingtontimes.com,Nancy Pelosi says she was ‘set up’ by San Fran...,House Speaker Nancy Pelosi said she was “set u...,
30805,1296922209890574337,2020-08-21 21:28:45,Liz_Cheney,R,Wyoming At-Large,U.S. Representative,RT @Kredo0: JUST IN: Congressional Republicans...,,https://freebeacon.com/national-security/congr...,freebeacon.com,Congressional Republicans Praise Trump Admin B...,A delegation of Republicans in the House and S...,
30806,1296082124970180613,2020-08-19 13:50:33,Liz_Cheney,R,Wyoming At-Large,U.S. Representative,Liz Cheney slams John Kerry's DNC remarks: Sur...,,https://www.foxnews.com/politics/liz-cheney-jo...,www.foxnews.com,Liz Cheney slams John Kerry's DNC remarks: Sur...,It is surprising that the Democratic National ...,


In [ ]:
#As we can see the data is unlabeled in terms clickbait or not. We know that we alredy have dataframe that has 
#a online article titeling regarding being click bait or not(clickbait.csv)
#We will use this labelled dataframe to train a logistic regression which we can 
#then use to predict the clickbait labels for our framing.p file/tweets.


In [ ]:
#So let us upload the labeled file using the code from the manual and have a first look at it by printing it. 
DATASET_URL = 'https://gist.githubusercontent.com/amitness/0a2ddbcb61c34eab04bad5a17fd8c86b/raw/66ad13dfac4bd1201e09726677dd8ba8048bb8af/clickbait.csv'
data = pd.read_csv(DATASET_URL)
data

,title,label
0,"15 Highly Important Questions About Adulthood,...",1
1,250 Nuns Just Cycled All The Way From Kathmand...,1
2,"Australian comedians ""could have been shot"" du...",0
3,Lycos launches screensaver to increase spammer...,0
4,Fußball-Bundesliga 2008–09: Goalkeeper Butt si...,0
...,...,...
31981,The Online Ad That Knows Where Your Friends Shop,0
31982,What Do You Love Based On Your Zodiac Sign,1
31983,This Baby's Reaction To Hearing About How She ...,1
31984,Saturn's moon Titan hosts liquid lakes and rivers,0


In [ ]:
#Now can split this labelled dataframe in train test data. We choose 20 % test since 
#this has already produced an accurate model as seen in the manual. 

In [ ]:
X = list(data.title.values)
y = list(data.label.values)# the labels we want to predict --> Y
labels = ['not clickbait', 'clickbait']

X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
#We use the Bag Of Words (BOW) to vectorize the text. We do not remove the stop words 
#since we have seen that some of them will later be classified as significant in classifying 
#a clickbait title. (E.g You and he)

In [ ]:
cv = CountVectorizer() # this initializes the CountVectorizer 

cv.fit(X_train_str) # create the vocabulary

X_train = cv.transform(X_train_str)
X_test = cv.transform(X_test_str)

In [ ]:
#We train the model with the vectorized text.

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='lbfgs')
lr.fit(X_train, y_train)

LogisticRegression()

In [ ]:
#we let it predict the cklickbait labels for each title from the test sest using predict()

In [ ]:
from sklearn.metrics import classification_report

y_pred = lr.predict(X_test)

In [ ]:
#We print out the accuracy and precision and the proportion of correct labels for each label category. 
print(classification_report(y_test, y_pred, 
                          target_names=labels))

               precision    recall  f1-score   support

not clickbait       0.96      0.98      0.97      3178
    clickbait       0.98      0.96      0.97      3220

     accuracy                           0.97      6398
    macro avg       0.97      0.97      0.97      6398
 weighted avg       0.97      0.97      0.97      6398



In [ ]:
#Another way to look at the performance of a classification model, is the so-called confusion matrix.
#Here, we can see how often a label is mistaken for another label

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred))

[[3112   66]
 [ 115 3105]]


In [ ]:
#I look at the words that are very typical for clickbait titles. 
#Good that we didnt filter out the stop words.Some of them are very typical for clickbait titles (e.g you,we, her)

In [ ]:
vocabulary = cv.get_feature_names()
regression_coefficients = lr.coef_[0] # get the LR weights
vocab_coef_combined = list(zip(regression_coefficients, vocabulary)) # this combines two separate lists [1, 2], ['word1', 'word2'] into one list [[1, 'word1'], [2, 'word2']]

feature_importance = pd.DataFrame(vocab_coef_combined,
                      columns=['coef', 'word'])
feature_importance.sort_values('coef', ascending=False).head(60)

,coef,word
20545,4.018000,you
18566,3.058204,these
18594,2.929925,this
154,2.922905,2015
2962,2.900514,buzzfeed
168,2.783366,21
8694,2.774251,here
12329,2.670787,my
18575,2.663407,things
20553,2.609577,your


In [ ]:
#Since we have a model with a high accuracy
#we can now predict the labels of the titles in the framing.p data frame.

In [ ]:

input = cv.transform(df['title']) #The title of our unlabelled df will be used as input 
y_pred = lr.predict(input) #to then predict the labels using the model we trained before


In [ ]:
y_pred #lets print them to see if it went well

array([1, 0, 0, ..., 0, 0, 0], dtype=int64)

In [ ]:
#We take the output and attach to the df unlabelled data frame. 

In [ ]:
df['labelclickbait'] = y_pred

In [ ]:
#Let us look at the df to see if the attachment worked. The new labels have been sucessfully added as column 'labelclickbait'.

In [ ]:
df.head(60) 

,tweet_id,date,user,party,state,chamber,tweet,news_mention,url_reference,netloc,title,description,label,labelclickbait
0,1325914751495499776,2020-11-09 21:34:45,SenShelby,R,Alabama,Senator,ICYMI – @BusinessInsider declared #Huntsville ...,businessinsider,https://www.businessinsider.com/personal-finan...,www.businessinsider.com,The 10 best US cities to move to if you want t...,The best US cities to move to if you want to s...,,1
1,1294021087118987264,2020-08-13 21:20:43,SenShelby,R,Alabama,Senator,Great news! Today @mazda_toyota announced an a...,,https://pressroom.toyota.com/mazda-and-toyota-...,pressroom.toyota.com,Mazda and Toyota Further Commitment to U.S. Ma...,"HUNTSVILLE, Ala., (Aug. 13, 2020) – Today, Maz...",,0
2,1323340848130609156,2020-11-02 19:06:59,DougJones,D,Alabama,Senator,He’s already quitting on the folks of Alabama ...,,https://apnews.com/article/c73f0dfe8008ebaf85e...,apnews.com,"Tuberville, Jones fight for Senate seat in Ala...","GARDENDALE, Ala. (AP) — U.S. Sen. Doug Jones, ...",,0
3,1323004075831709698,2020-11-01 20:48:46,DougJones,D,Alabama,Senator,I know you guys are getting bombarded with fun...,,https://secure.actblue.com/donate/djfs-close?r...,secure.actblue.com,I just gave!,Join us! Contribute today.,negiotated,1
4,1322567531320717314,2020-10-31 15:54:06,DougJones,D,Alabama,Senator,"Well looky here folks, his own players don’t t...",,https://slate.com/culture/2020/10/tommy-tuberv...,slate.com,What Tommy Tuberville’s Former Auburn Players ...,"""All I could think is, why?""",,0
6,1320853775893094401,2020-10-26 22:24:15,DougJones,D,Alabama,Senator,Thanks so much guys. Really enjoyed being wit...,,http://dougjones.com,dougjones.com,One Alabama,"Alabama, like America, is a dynamic and divers...",,0
8,1319698596074262530,2020-10-23 17:53:58,DougJones,D,Alabama,Senator,"We are excited to have @DebraMessing, @SeanHay...",,http://secure.actblue.com/donate/10.23.20wg,secure.actblue.com,I just gave!,Join us! Contribute today.,,1
9,1319690481501114368,2020-10-23 17:21:44,DougJones,D,Alabama,Senator,Try this link: https://t.co/TuqKYnXhsX http...,,https://secure.actblue.com/donate/10.23.20wg,secure.actblue.com,I just gave!,Join us! Contribute today.,,1
10,1319090524612141061,2020-10-22 01:37:43,DougJones,D,Alabama,Senator,"Chris, we're so close but with friends like yo...",,https://secure.actblue.com/donate/csm-bes-jone...,secure.actblue.com,Stand with Doug Jones,Make a donation to Doug Jones today to make su...,,0
11,1318544465146109954,2020-10-20 13:27:52,DougJones,D,Alabama,Senator,The answer to the headline is “No” but read th...,,http://al.com,al.com,"Alabama Local News, Breaking News, Sports & We...","Get the latest Alabama Local News, Sports News...",,0


In [ ]:
df.tail(60)

,tweet_id,date,user,party,state,chamber,tweet,news_mention,url_reference,netloc,title,description,label,labelclickbait
30735,1306367488368824321,2020-09-16 23:00:55,TomTiffanyWI,R,Wisconsin 7th District,U.S. Representative,??? I’ll just say this… Thank God for Presiden...,,https://www.wsaw.com/2020/09/09/evers-has-no-r...,www.wsaw.com,Evers has âno regretsâ about response to K...,Gov. Tony Evers says he has âno regretsâ a...,,1
30736,1305919332514496513,2020-09-15 17:20:06,TomTiffanyWI,R,Wisconsin 7th District,U.S. Representative,Congratulations to UW-Superior for their recen...,,https://www.wdio.com/news/great-lakes-grants-s...,www.wdio.com,Great Lakes grants to help study ballast water...,A $5 million grant to the Lake Superior Resear...,,0
30737,1305189802208301057,2020-09-13 17:01:12,TomTiffanyWI,R,Wisconsin 7th District,U.S. Representative,All hands on deck! Help show Democrats that Wi...,,https://tomtiffany.com/volunteer/,tomtiffany.com,Volunteer - Tom Tiffany for Congress,,,0
30738,1304789467589378048,2020-09-12 14:30:25,TomTiffanyWI,R,Wisconsin 7th District,U.S. Representative,"You, me and President Trump built the world’s ...",,https://thehill.com/policy/finance/515074-augu...,thehill.com,"Economy adds 1.4 million jobs in August, unemp...","The U.S. added 1.4 million jobs in August, the...",,0
30740,1303717427910504448,2020-09-09 15:30:31,TomTiffanyWI,R,Wisconsin 7th District,U.S. Representative,My first 100 days as your representative has f...,,https://www.wsaw.com/2020/08/27/100-days-in-of...,www.wsaw.com,100 days in office: Rep. Tom Tiffanyâs recor...,Wednesday marked 100 days in office for freshm...,,0
30741,1302360873969881091,2020-09-05 21:40:03,TomTiffanyWI,R,Wisconsin 7th District,U.S. Representative,Want to help make sure Republicans get elected...,,https://tomtiffany.com/volunteer/,tomtiffany.com,Volunteer - Tom Tiffany for Congress,,,0
30742,1302296433077506050,2020-09-05 17:23:59,TomTiffanyWI,R,Wisconsin 7th District,U.S. Representative,Next Mr Biden will comment on Benghazi. Then s...,,https://apnews.com/8f31d7343274c14e02432a7d57c...,apnews.com,Biden slams Trump over alleged comments mockin...,WASHINGTON (AP) — Democratic presidential nomi...,negiotated,0
30743,1301530427304312833,2020-09-03 14:40:09,TomTiffanyWI,R,Wisconsin 7th District,U.S. Representative,Want to help conservatives get elected this fa...,,https://tomtiffany.com/volunteer/editor/,tomtiffany.com,Send Letter To The Editor - Tom Tiffany for Co...,,,0
30744,1300063396012974080,2020-08-30 13:30:42,TomTiffanyWI,R,Wisconsin 7th District,U.S. Representative,Great seeing this local church step up for our...,,https://waow.com/2020/08/22/food-pantrys-filli...,waow.com,Food Pantry's filling needs In Wausau,"Wausau, Wis. (WAOW) -- Thrive Church in Wausau...",,0
30746,1297979322259644417,2020-08-24 19:29:20,TomTiffanyWI,R,Wisconsin 7th District,U.S. Representative,I’ll be tuning into the Republican Convention ...,,https://www.2020gopconvention.com/blog/where-t...,www.2020gopconvention.com,"Where to Watch on Monday, August 24th, 2020 - ...",2020 Republican National Convention,,0


In [ ]:
#How many times do democrats refer to an article with a clickbait title and how many times do republicans do? 

#For that we look at the dataframe that shows clickbait titles used by the Democratic Party. We count the rows to the get the number.

In [ ]:
countD1 = df[(df['labelclickbait'] == 1) & (df['party'] =='D' )].shape[0]
countD1

2830

In [ ]:
#we get 2830 rows where democrats refer to an article with a title labelled by us a clickbait.
#Now we count the same for Republicans.

In [ ]:
countR1=df[(df['labelclickbait'] == 1) & (df['party'] =='R' )].shape[0]
countR1

1058

In [ ]:
#we get 1058 rows where republicans refer to an article with a title labelled by us a clickbait.

In [ ]:
CounttotalR = df[df['party'] =='R'].shape[0]
CounttotalD = df[df['party'] =='D'].shape[0]


In [ ]:
#lets look at the percentage as compared to total amounts of tweets per party.

In [ ]:
PercD = (countD1/CounttotalD)*100
PercD

18.182986378822925

In [ ]:
PercR = (countR1/CounttotalR)*100
PercR

13.61472140007721

In [ ]:
#We can also see that percentage wise (clickbait title of all post by this party) the Democrats refer to clickbait articles  more tham Republicans.Namely 4% percent more. 

In [ ]:
#To see if this difference is statistically significant we can do a CHi Square test to see if this difference is statistically significant.
#We first drop the I and L parrties from the dataframe, then make a contgency table based upon which we perform our chi square test.

In [ ]:
partyclickbaitcrosstab = pd.crosstab(df[(df['party'] =='D') | (df['party'] =='R' )]['labelclickbait'],
                                     df[(df['party'] =='D') | (df['party'] =='R' )]['party'], margins = False)
partyclickbaitcrosstab 

party,D,R
labelclickbait,,
0,12734,6713
1,2830,1058


In [ ]:
#Using this table we can now perform this Chi Square Test. 

In [ ]:
stat, p, dof, expected = chi2_contingency(partyclickbaitcrosstab) 
 

In [ ]:
alpha = 0.05
print("p value is " + str(p)) 
if p <= alpha: 
    print('Dependent (reject H0)') 
else: 
    print('Independent (H0 holds true)') 

p value is 1.2810427655490298e-18
Dependent (reject H0)


In [ ]:
#the null hypothesis is that there is no significant difference between proportions in the labelled posts of Democarats and Republicans. This can be rejected and supports that this difference is statiscally significant. 

In [ ]:

dfclickbait = df[df['labelclickbait'] == 1] #allthe clickbait articles for a brief look 
dfclickbait

,tweet_id,date,user,party,state,chamber,tweet,news_mention,url_reference,netloc,title,description,label,labelclickbait
0,1325914751495499776,2020-11-09 21:34:45,SenShelby,R,Alabama,Senator,ICYMI – @BusinessInsider declared #Huntsville ...,businessinsider,https://www.businessinsider.com/personal-finan...,www.businessinsider.com,The 10 best US cities to move to if you want t...,The best US cities to move to if you want to s...,,1
3,1323004075831709698,2020-11-01 20:48:46,DougJones,D,Alabama,Senator,I know you guys are getting bombarded with fun...,,https://secure.actblue.com/donate/djfs-close?r...,secure.actblue.com,I just gave!,Join us! Contribute today.,negiotated,1
8,1319698596074262530,2020-10-23 17:53:58,DougJones,D,Alabama,Senator,"We are excited to have @DebraMessing, @SeanHay...",,http://secure.actblue.com/donate/10.23.20wg,secure.actblue.com,I just gave!,Join us! Contribute today.,,1
9,1319690481501114368,2020-10-23 17:21:44,DougJones,D,Alabama,Senator,Try this link: https://t.co/TuqKYnXhsX http...,,https://secure.actblue.com/donate/10.23.20wg,secure.actblue.com,I just gave!,Join us! Contribute today.,,1
19,1311091680402079745,2020-09-29 23:53:10,DougJones,D,Alabama,Senator,"Proud to have the endorsement of my friend, th...",,http://vote.org,vote.org,Everything You Need to Vote - Vote.org,Register to vote. Check your registration stat...,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30771,1298723241284927488,2020-08-26 20:45:24,SenatorEnzi,R,Wyoming,U.S. Senator,"RT @TRUCKINGdotORG: Next week, four female @Am...",,https://www.trucking.org/news-insights/louisa-...,www.trucking.org,Louisa Swain and the Trailblazers Who Show Us ...,The significance of this year’s national elect...,,1
30776,1309157440357904392,2020-09-24 15:47:11,SenJohnBarrasso,R,Wyoming,U.S. Senator,"RT @Interior: #OTD in 1906, President Theodore...",,http://www.sharetheexperience.org,www.sharetheexperience.org,Share the Experience Photo Contest,Upload your photo of Federal Lands for a chanc...,,1
30786,1294645816196562944,2020-08-15 14:43:10,SenJohnBarrasso,R,Wyoming,U.S. Senator,Wrote in @FoxNews about the coming #coronaviru...,FoxNews,https://www.foxnews.com/opinion/coronavirus-va...,www.foxnews.com,Sen. John Barrasso: A coronavirus vaccine is c...,America is close to a coronavirus vaccine. It ...,affirmative,1
30787,1324364432315658240,2020-11-05 14:54:20,Liz_Cheney,R,Wyoming At-Large,U.S. Representative,RT @cnsnews: “The surge of victorious pro-life...,cnsnews,https://www.cnsnews.com/article/washington/pat...,www.cnsnews.com,New Pro-Life Republican Congresswomen in House...,The number of pro-life Republican women in the...,,1


In [ ]:
dfclickbait.shape[0] #the amount of clickbait articles

3909

In [ ]:
dfD =df[df['party'] == 'D'] 
dfD.shape[0] #the amount of titles for the Democrats

15564

In [ ]:
dfR=df[df['party'] == 'R'] 
dfR.shape[0] #the amount of titles for the Republicans

7771

In [ ]:
dfclickbaitD =dfclickbait[dfclickbait['party'] == 'D'] #all the democrat clickbait titles
dfclickbaitD
dfclickbaitD.shape[0] 

2830

In [ ]:
dfclickbaitR =dfclickbait[dfclickbait['party'] == 'R'] #all the republicans clickbait titles
dfclickbaitR 
dfclickbaitR.shape[0]

1058

In [ ]:
#we nwo go thorugh some popular topics to see how they compare for each party

In [ ]:
dfclickbaitD[dfclickbaitD['title'].str.contains('Covid', na = False)].shape[0] 

118

In [ ]:
dfclickbaitD[dfclickbaitD['title'].str.contains('Corona', na = False)].shape[0]

44

In [ ]:
dfclickbaitR[dfclickbaitR['title'].str.contains('Covid', na = False)].shape[0]

9

In [ ]:
dfclickbaitR[dfclickbaitR['title'].str.contains('Corona', na = False)].shape[0]

32

In [ ]:
dfclickbaitD[dfclickbaitD['title'].str.contains('just gave', na = False)].shape[0]

23

In [ ]:
dfclikbaitR[dfclickbaitR['title'].str.contains('just gave', na = False)].shape[0]

1

In [ ]:
dfclickbaitD[dfclickbaitD['title'].str.contains('vote', na = False)].shape[0]

77

In [ ]:
dfclickbaitR[dfclickbaitR['title'].str.contains('vote', na = False)].shape[0]

22

In [ ]:
#lets look at the news sources used by the different parties and their clickbait proportions

In [ ]:
df.netloc.value_counts()

www.nytimes.com           1508
                          1092
www.politico.com           623
www.cnn.com                602
thehill.com                550
                          ... 
immigrantreliefwa.org        1
winningforwomen.com          1
help304.com                  1
VoteInNH.org                 1
allianceportregion.com       1
Name: netloc, Length: 3602, dtype: int64

In [ ]:
dfclickbait.netloc.value_counts()

www.nytimes.com                   312
www.nbcnews.com                   132
www.cnn.com                       117
www.theatlantic.com               114
www.foxnews.com                   106
                                 ... 
wrex.com                            1
www.yaleclimateconnections.org      1
northdallasgazette.com              1
spectrumnews1.com                   1
www.tnjustice.org                   1
Name: netloc, Length: 911, dtype: int64

In [ ]:
dfD.netloc.value_counts()

www.nytimes.com               1420
                               575
www.cnn.com                    563
www.politico.com               555
www.nbcnews.com                456
                              ... 
allinwa.org                      1
www.signupgenius.com             1
festival.texastribune.org        1
www.washingtoninformer.com       1
www.pinknews.co.uk               1
Name: netloc, Length: 2378, dtype: int64

In [ ]:
dfR.netloc.value_counts()

www.foxnews.com                  535
                                 511
thehill.com                      186
www.cnbc.com                     146
www.foxbusiness.com              126
                                ... 
cutoutoverdoses.com                1
www.nam.org                        1
hellowoodlands.com                 1
www.mcknightsseniorliving.com      1
www.falloncountyextra.com          1
Name: netloc, Length: 1727, dtype: int64

In [ ]:
dfclickbaitD.netloc.value_counts()

www.nytimes.com                     302
www.nbcnews.com                     128
www.theatlantic.com                 110
www.cnn.com                         107
nyti.ms                              77
                                   ... 
rssfeeds.tennessean.com               1
DemConvention.com                     1
www.hrc.org                           1
Ballotform.Philadelphiavotes.com      1
www.hachettebookgroup.com             1
Name: netloc, Length: 643, dtype: int64

In [ ]:
dfclickbaitR.netloc.value_counts()

www.foxnews.com           103
thehill.com                48
www.nationalreview.com     32
fxn.ws                     30
podcasts.apple.com         27
                         ... 
thetandd.com                1
bigten.org                  1
www.dcourier.com            1
cs.pn                       1
t.uga.edu                   1
Name: netloc, Length: 367, dtype: int64

# SUMMARY

We have seen that using a logistic regression as classifier that Democrats refer to Clickbait inn 18 % percent of all their tweets whereas Republicans do so only for 14%. 
Moreover, the difference Is significant. 
In total numbers Democrats have 1722 more clickbait title. Intuitively, one might that think that republicans refer more to clickbait articles. 

Whymore intuitive? Because the stemvoters of the republicanparty are majorly working class and thus often less educated. Classic clickbait titles such as: “Find out how you can make 4000 Dollars from home” might be more attractive to less educated people.

A look at the first 60 rows and titles of our newly labelled data frames gives us a hint why democrats could proprtionally more cb articles. There are many clickbait articles by the democrats with the same title. “I just gave”. This refers to a donation campaign by the democrats after Justice Ruth Bader Ginsburg’s death was announced where they reached a record-number of donations, as seen here also with the help of twitter.

https://www.nytimes.com/2020/09/18/us/elections/democrats-shatter-actblues-donation-records-after-ginsburgs-death.html

In fact if we look at the amount of clickbait titles with this type among democrats, we can see that they constitute for 23 clickbait titles among democrats
There is also a striking difference between Clickbait articles related to the Coronavirus between  Democrats and Republicans. This is line with reports suggesting that Democrats care more about the Coronavirus than Republicans
Democats also gave much more Clickbait insructions on “how to vote”.

Lastly,we saw that New York Times which is mainly cited by the Democrats accounts for 8% and the most of all clickbait articles (312 of a total of 3909 clickbait titles)which could be a main driver for significant difference of clickbait titles proportians between the two parties. A similar pattern can be obeserved for CNN. Politico.com a popular news source by the Republicans does not seem be very clickbaity.



